## Sample Notebook for making predictions of mlcsv_future data

In [1]:
import numpy as np
import pandas as pd
from getpass import getpass
from sqlalchemy import create_engine,Table, MetaData, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
import sqlalchemy_utils
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
import pyodbc 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow import keras


### Read Test Future data into DataFrame

In [2]:
mlcsv_future_df = pd.read_csv('../csv Data/test_neo_public_mlcsv_future.csv')

In [3]:
mlcsv_future_df.head()

,spkid,full_name,pdes,h,orbit_id,epoch,epoch.mjd,epoch.cal,e,a,...,orbit_id_cad,jd,cd,dist,dist_min,dist_max,v_rel,v_inf,t_sigma_f,h_cad
0,2000433,433 Eros (A898 PA),433,10.43,JPL 659,2459600.5,59600,2022-01-21.0,0.2227,1.458,...,659,2.485544e+06,2093-01-31 15:47:00.000000,0.182464,0.182464,0.182464,5.974914,5.972469,< 00:01,10.43
1,2000887,887 Alinda (A918 AA),887,13.87,JPL 428,2459600.5,59600,2022-01-21.0,0.5705,2.473,...,429,2.483346e+06,2087-01-25 21:50:00.000000,0.166325,0.166321,0.166329,11.268948,11.267526,< 00:01,13.87
2,2001221,1221 Amor (1932 EA1),1221,17.39,JPL 110,2459600.5,59600,2022-01-21.0,0.4358,1.919,...,110,2.487064e+06,2097-03-31 04:56:00.000000,0.140528,0.140527,0.140528,7.564784,7.562277,< 00:01,17.39
3,2001566,1566 Icarus (1949 MA),1566,16.35,JPL 126,2459600.5,59600,2022-01-21.0,0.8269,1.078,...,126,2.484582e+06,2090-06-14 07:08:00.000000,0.043536,0.043529,0.043543,28.169786,28.167613,< 00:01,16.35
4,2001620,1620 Geographos (1951 RA),1620,15.29,JPL 649,2459600.5,59600,2022-01-21.0,0.3354,1.246,...,652,2.486124e+06,2094-09-03 10:34:00.000000,0.190024,0.190023,0.190025,16.694264,16.693424,< 00:01,15.29


In [4]:
# Create new dataframe with descriptive meta-data column for Visualization
descr_future_df = mlcsv_future_df.loc[:, ['spkid', 'full_name', 'pdes', 'class', 'first_obs', 'last_obs', 'n_obs_used']]
descr_future_df


,spkid,full_name,pdes,class,first_obs,last_obs,n_obs_used
0,2000433,433 Eros (A898 PA),433,AMO,1893-10-29,2021-05-13,9130
1,2000887,887 Alinda (A918 AA),887,AMO,1918-02-03,2021-07-31,2624
2,2001221,1221 Amor (1932 EA1),1221,AMO,1932-03-12,2021-12-05,573
3,2001566,1566 Icarus (1949 MA),1566,APO,1949-07-01,2021-08-11,1078
4,2001620,1620 Geographos (1951 RA),1620,APO,1951-09-14,2022-01-01,4724
...,...,...,...,...,...,...,...
495,2172722,172722 (2004 BV102),172722,APO,2004-01-31,2020-04-02,469
496,2173561,173561 (2000 YV137),173561,APO,2000-12-01,2021-12-03,642
497,2173664,173664 (2001 JU2),173664,AMO,2001-05-15,2021-01-18,263
498,2174050,174050 (2002 CC19),174050,AMO,1986-02-07,2021-09-11,759


#### Define list of features for making future predictions


In [5]:
# Define feature columns to subset dataframe
feature_columns = ["moid","moid_ld","h","v_rel","v_inf"]

In [7]:
output_df = mlcsv_future_df.loc[:, feature_columns]

In [8]:
output_df.dtypes

moid       float64
moid_ld    float64
h          float64
v_rel      float64
v_inf      float64
dtype: object

In [ ]:
# Extract feature variables from future data set
X = output_df


In [ ]:
# Create a StandardScaler instances for feature variabbles
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X)

# Scale the data
X_predict_scaled = X_scaler.transform(X)


In [ ]:
# Load the Neural Networks Model "nn"
forecast_model = tf.keras.models.load_model('NEOs_project_NN.h5')


In [ ]:
# Evaluate the forecast model using the future data
predictions = forecast_model.evaluate(feed_dict = {X_predict_scaled})

In [ ]:
# list of predictions
predictions = []
type(predictions)

In [ ]:
# iterate through mlcsv future dataframe to get model preduction for each asteroid close approach  
for row in output_df.iterrows():
    prediction = get_classifier_prediction()
    predictions.append(prediction)

In [ ]:
# Create Pandas Series from predictions list
prediction_series = pd.Series(predictions)

In [ ]:
# Add new prediction column to dataframe
output_df['PHA_prediction'] = prediction_series

In [ ]:
# Merge output dataframe with prediction column to the descr_future_df dataframe
new_output_df = pd.merge(descr_future_df, output_df, how='inner', left_index=True, right_index=True)

In [ ]:
new_output_df['PHA_prediction'].value_counts()

In [ ]:
new_output_df.head()

In [ ]:
new_output_df.to_csv('test_future_predictions_data.csv', index=False)